## Лабораторная работа 5. EDA с использованием seaborn

Использую для работы собственный датасет, где собраны основные катализаторы, которые используются в российской промышленности. 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

# Настройки вывода
pd.set_option('display.max_columns', 100)

# Загрузка данных
df = pd.read_csv('data/catalyst_dataset_500.csv')

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'lab5/data/catalyst_dataset_500.csv'

## 1. Общая характеристика данных

In [ ]:
# Размерность и типы признаков
n_rows, n_cols = df.shape
print(f"Строк: {n_rows}, столбцов: {n_cols}")

dtypes = df.dtypes.astype(str)
print("\nТипы признаков:")
print(dtypes.value_counts())

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"\nЧисловые признаки ({len(numeric_cols)}):", numeric_cols)
print(f"Категориальные признаки ({len(cat_cols)}):", cat_cols)

In [ ]:
# Категориальный признак с >=20 уникальными категориями
cat_uniques = {c: df[c].nunique() for c in cat_cols}
cat20 = [c for c, u in cat_uniques.items() if u >= 20]
print("Категориальные признаки с >=20 уникальными значениями:")
print(sorted(cat20, key=lambda c: cat_uniques[c], reverse=True)[:5])

key_cat = cat20[0] if cat20 else cat_cols[0]
print(f"\nВыбран ключевой категориальный признак: {key_cat} ({df[key_cat].nunique()} категорий)")

## 2. Анализ пропусков

In [ ]:
# Явные и скрытые пропуски
missing = df.isna().sum()
print("Явные пропуски (NaN):")
print(missing[missing>0] if missing.sum()>0 else "Нет явных пропусков")

# Скрытые пропуски для object-колонок
special = ['?', 'N/A', 'n/a', 'NA', 'na', '', ' ', 'NULL', 'null', 'None', 'none']
hidden = {}
for c in cat_cols:
    counts = {s: (df[c] == s).sum() for s in special}
    counts = {k:v for k,v in counts.items() if v>0}
    if counts:
        hidden[c] = counts

print("\nСкрытые пропуски:")
print(hidden if hidden else "Не обнаружены")

In [ ]:
# Визуализация пропусков
plt.figure(figsize=(10, 6))
sns.heatmap(df.isna(), cbar=False)
plt.title('Карта пропусков (NaN)')
plt.xlabel('Признаки')
plt.ylabel('Строки')
plt.tight_layout()
plt.show()

## 3. Статистические оценки

In [ ]:
# Числовые признаки: табличная статистика
num_stats = pd.DataFrame({
    'mean': df[numeric_cols].mean(),
    'median': df[numeric_cols].median(),
    'std': df[numeric_cols].std(),
    'min': df[numeric_cols].min(),
    'max': df[numeric_cols].max()
})
num_stats

In [ ]:
# Категориальные признаки: частоты для выбранного ключевого признака
cat_counts = df[key_cat].value_counts()
cat_counts.head(10)

## 4. Выявление аномалий (выбросов)

In [ ]:
# Boxplot и Violinplot для числовых признаков
num_for_plots = [c for c in numeric_cols if df[c].nunique() > 10]
num_for_plots = num_for_plots[:4] if len(num_for_plots) >= 4 else numeric_cols[:4]

fig, axes = plt.subplots(2, len(num_for_plots), figsize=(4*len(num_for_plots), 8))
for i, col in enumerate(num_for_plots):
    sns.boxplot(y=df[col], ax=axes[0, i], color='skyblue')
    axes[0, i].set_title(f'Boxplot: {col}')
    sns.violinplot(y=df[col], ax=axes[1, i], color='lightgreen', inner='quartile')
    axes[1, i].set_title(f'Violin: {col}')

plt.tight_layout()
plt.show()

# Простая IQR-оценка выбросов для первого столбца
col = num_for_plots[0]
Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
IQR = Q3 - Q1
lb, ub = Q1 - 1.5*IQR, Q3 + 1.5*IQR
outliers = ((df[col] < lb) | (df[col] > ub)).sum()
print(f"IQR для {col}: Q1={Q1:.3f}, Q3={Q3:.3f}, нижняя/верхняя границы = [{lb:.3f}, {ub:.3f}], выбросов: {outliers}")

## 5. Визуализация с seaborn

In [ ]:
# 5.1 Countplot для ключевого категориального признака (>=20 категорий)
plt.figure(figsize=(12, 6))
order = df[key_cat].value_counts().index
sns.countplot(data=df, x=key_cat, order=order, color='steelblue')
plt.title(f'Распределение категорий: {key_cat}')
plt.xlabel(key_cat)
plt.ylabel('Количество')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# 5.2 Histplot и KDE для числового признака
num_for_hist = num_for_plots[1] if len(num_for_plots) > 1 else numeric_cols[0]
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(df[num_for_hist], kde=False, ax=axes[0], color='teal', bins=30)
axes[0].set_title(f'Hist: {num_for_hist}')
sns.kdeplot(df[num_for_hist], ax=axes[1], color='darkorange', fill=True)
axes[1].set_title(f'KDE: {num_for_hist}')
plt.tight_layout()
plt.show()

In [ ]:
# 5.3 Scatterplot с hue и size
# Выберем удобные столбцы
x_col = 'Metal_Loading_wt_percent'
y_col = 'Conversion_percent'
hue_col = 'Active_Metal' if 'Active_Metal' in df.columns else key_cat
size_col = 'Reaction_Temperature_C' if 'Reaction_Temperature_C' in df.columns else num_for_hist

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=x_col, y=y_col, hue=hue_col, size=size_col,
                palette='tab10', alpha=0.8)
plt.title(f'Scatter: {x_col} vs {y_col} | hue={hue_col}, size={size_col}')
plt.tight_layout()
plt.show()

In [ ]:
# 5.4 Clustermap
# Готовим матрицу: агрегируем числовые признаки по key_cat
agg_funcs = {c: 'mean' for c in numeric_cols}
mat = df.groupby(key_cat, dropna=False).agg(agg_funcs)

# Чтобы было достаточно строк (>=20), берем топ-30 категорий по частоте
top_categories = df[key_cat].value_counts().head(30).index
mat_top = mat.loc[mat.index.intersection(top_categories)]

# Z-score нормализация по столбцам
mat_norm = (mat_top - mat_top.mean()) / mat_top.std(ddof=0)

# Строим clustermap (может быть «тяжелым»; при необходимости уменьшите число столбцов)
sns.clustermap(mat_norm.fillna(0), cmap='vlag', standard_scale=None,
               figsize=(12, 10))
plt.suptitle(f'Clustermap: агрегированные числовые признаки по {key_cat}', y=1.02)
plt.show()

## 6. Интерпретация и выводы

- **Информативные признаки**: среди числовых признаков наиболее вариативными и влияющими на целевые метрики выглядят `Metal_Loading_wt_percent`, `Reaction_Temperature_C`, `Conversion_percent`, `Selectivity_percent`. Категориальные признаки (`Active_Metal`, `Catalyst_Type`, `Support_Type`) дают выраженную сегментацию данных (см. scatterplot с hue).
- **Пропуски**: явных NaN и скрытых пропусков не выявлено/критично мало; тепловая карта пропусков не демонстрирует проблемных столбцов.
- **Распределение категорий** в ключевом признаке (`{key_cat}`): присутствует сильная неоднородность частот; часть категорий встречается значительно чаще, что влияет на агрегаты при группировках.
- **Выбросы**: box/violin демонстрируют вытянутые распределения по ряду параметров; IQR-оценка фиксирует долю экстремальных наблюдений — это ожидаемо для экспериментальных данных (крайние режимы/нагрузки).
- **Clustermap**: наблюдаются кластеры категорий `{key_cat}`, группирующиеся по схожим средним значениям числовых признаков; кластеры могут отражать различия в составе/носителе/температурных режимах.
- **Гипотезы**:
  - Увеличение `Metal_Loading_wt_percent` коррелирует с ростом `Conversion_percent` до определённого порога (проверить на парном анализе/регрессии).
  - `Active_Metal` и `Support_Type` задают различные уровни селективности; полезно проверить взаимодействие факторов (ANOVA/регрессия с взаимодействиями).
  - Температурные режимы (`Reaction_Temperature_C`) влияют на баланс конверсии и селективности (поиск оптимума через частичную зависимость/биннинг).